In [2]:
# We use the pdfplumber library to open the PDF and extract text from each page.
import pdfplumber
# re for regular expression for pattern matching
import re 
import pandas as pd
# Assigning file location
invoice_file = r"C:/Users/MANISH KUMAR/Desktop/Untitled Folder/Amazon_invoice4.pdf"

In [3]:
# Extract raw data from pdf invoice

with pdfplumber.open(invoice_file) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
text

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


'Tax Invoice/Bill of Supply/Cash Memo\n(Original for Recipient)\nSold By : Billing Address :\nCAMBIUM RETAIL PRIVATE LIMITED sakthivel\n* GMR Airport City, Survey No. 99/1, Mamidipally 42-keela vaikolkara street ,kulathu sandhu,\nVillage, Shamshabad woraiyur\nHyderabad, Telangana, 500108 Trichy, TAMIL NADU, 620003\nIN IN\nState/UT Code: 33\nPAN No: AAMCA5258P\nGST Registration No: 36AAMCA5258P1ZY Shipping Address :\nsakthivel\nsakthivel\n42-keela vaikolkara street ,kulathu sandhu,\nworaiyur\nTrichy, TAMIL NADU, 620003\nIN\nState/UT Code: 33\nPlace of supply: TAMIL NADU\nPlace of delivery: TAMIL NADU\nOrder Number: 171-0472773-6395537 Invoice Number : IN-HYD8-1939\nOrder Date: 15.06.2020 Invoice Details : TG-HYD8-154003231-2021\nInvoice Date : 16.06.2020\nSl. Unit Net Tax Tax Tax Total\nDescription DiscountQty\nNo Price AmountRateTypeAmountAmount\n1 (Renewed) PTron Soundster Bluetooth Headphones, Over\nThe Ear, Wireless Earphones with Mic, High Bass, 20 Hours ₹824.58 ₹0.00 1 ₹824.58 18%

In [4]:
# split lines end with \
# page.extract_text() extracts the raw text from each page

with pdfplumber.open(invoice_file) as pdf:
    page = pdf.pages[0]
    text = page.extract_text()
    for line in text.split('\n'):
        print(line)
    

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Tax Invoice/Bill of Supply/Cash Memo
(Original for Recipient)
Sold By : Billing Address :
CAMBIUM RETAIL PRIVATE LIMITED sakthivel
* GMR Airport City, Survey No. 99/1, Mamidipally 42-keela vaikolkara street ,kulathu sandhu,
Village, Shamshabad woraiyur
Hyderabad, Telangana, 500108 Trichy, TAMIL NADU, 620003
IN IN
State/UT Code: 33
PAN No: AAMCA5258P
GST Registration No: 36AAMCA5258P1ZY Shipping Address :
sakthivel
sakthivel
42-keela vaikolkara street ,kulathu sandhu,
woraiyur
Trichy, TAMIL NADU, 620003
IN
State/UT Code: 33
Place of supply: TAMIL NADU
Place of delivery: TAMIL NADU
Order Number: 171-0472773-6395537 Invoice Number : IN-HYD8-1939
Order Date: 15.06.2020 Invoice Details : TG-HYD8-154003231-2021
Invoice Date : 16.06.2020
Sl. Unit Net Tax Tax Tax Total
Description DiscountQty
No Price AmountRateTypeAmountAmount
1 (Renewed) PTron Soundster Bluetooth Headphones, Over
The Ear, Wireless Earphones with Mic, High Bass, 20 Hours ₹824.58 ₹0.00 1 ₹824.58 18% IGST ₹148.42 ₹973.00
of Mus

In [5]:
# Normalize text: flatten newlines and multiple spaces
normalized_text = ' '.join(text.split())
print(normalized_text)

Tax Invoice/Bill of Supply/Cash Memo (Original for Recipient) Sold By : Billing Address : CAMBIUM RETAIL PRIVATE LIMITED sakthivel * GMR Airport City, Survey No. 99/1, Mamidipally 42-keela vaikolkara street ,kulathu sandhu, Village, Shamshabad woraiyur Hyderabad, Telangana, 500108 Trichy, TAMIL NADU, 620003 IN IN State/UT Code: 33 PAN No: AAMCA5258P GST Registration No: 36AAMCA5258P1ZY Shipping Address : sakthivel sakthivel 42-keela vaikolkara street ,kulathu sandhu, woraiyur Trichy, TAMIL NADU, 620003 IN State/UT Code: 33 Place of supply: TAMIL NADU Place of delivery: TAMIL NADU Order Number: 171-0472773-6395537 Invoice Number : IN-HYD8-1939 Order Date: 15.06.2020 Invoice Details : TG-HYD8-154003231-2021 Invoice Date : 16.06.2020 Sl. Unit Net Tax Tax Tax Total Description DiscountQty No Price AmountRateTypeAmountAmount 1 (Renewed) PTron Soundster Bluetooth Headphones, Over The Ear, Wireless Earphones with Mic, High Bass, 20 Hours ₹824.58 ₹0.00 1 ₹824.58 18% IGST ₹148.42 ₹973.00 of Mus

# Extracting all details by one hard incoding

In [7]:
normalized = ' '.join(text.split())

# Patterns for extraction
patterns = {
    "Order Number": r"Order\s*Number\s*:\s*([\d-]+)",
    "Invoice Number": r"Invoice\s*Number\s*:\s*([A-Z0-9-]+)",
    "Order Date": r"Order\s*Date\s*:\s*([\d./]+)",
    "Invoice Date": r"Invoice\s*Date\s*:\s*([\d./]+)",
    "Invoice Details": r"Invoice\s*Details\s*:\s*([A-Z0-9-]+)",
    "PAN Number": r"PAN\s*No\s*:\s*([A-Z0-9]+)",
    "GST Number": r"GST\s*Registration\s*No\s*:\s*([A-Z0-9]+)",
    "Amount in Words": r"Amount in Words:\s*([A-Za-z\s\-]+only)",
    "Total Amount": r"TOTAL:\s*₹[\d,.]+\s*₹([\d,.]+)"
}

# Extract patterns
results = {}
for key, pattern in patterns.items():
    match = re.search(pattern, normalized, re.IGNORECASE)
    results[key] = match.group(1) if match else "Not found"
    
# Extract block between "Shipping Address :" and "Order Number"
shipping_match = re.search(r"Shipping Address\s*:(.*?)Order Number", text, re.DOTALL | re.IGNORECASE)
shipping_block = shipping_match.group(1).strip() if shipping_match else " "

# Remove PAN and GST numbers only (but preserve other text like names)
shipping_block = re.sub(r'PAN No:\s*\w+\s*', '', shipping_block)
shipping_block = re.sub(r'GST Registration No:\s*\w+\s*', '', shipping_block)

# Clean up extra spaces
shipping_address_cleaned = ' '.join(shipping_block.split())
print("Extracted Shipping Address:\n", shipping_address_cleaned)
    
# Regular expression to extract Billing Address
match = re.search( r'Billing Address\s*:(.*?)Shipping Address', text, re.DOTALL |  re.IGNORECASE)
Billing_Address = match.group(1) if match else None

print("Extracted Billing Address    :    ", Billing_Address)    
    
# Isolate the product section
product_section_match = re.search(r'(Sl\..*?)(Amount in Words:)', text, re.DOTALL)
product_section = product_section_match.group(1) if product_section_match else ""
# Normalize whitespace
product_section_cleaned = ' '.join(product_section.split())

#Extract the product description before the first pipe "|"
product_match = re.search(r'\d+\s+(.*?)\s+\|', product_section_cleaned)
product_details = product_match.group(1).strip() if product_match else None

print("Extracted Product_Details    :    ", product_details)   

# Add addresses and product details to results
results["Billing Address"] = Billing_Address
results["Shipping Address"] = shipping_address_cleaned
results["Product Details"] = product_details
    
df = pd.DataFrame(list(results.items()), columns=["Field", "Detail"])

# Save to Excel
df.to_excel("Amazon_invoice4_data_extracted.xlsx", index=False, engine='openpyxl')

print("Data saved to 'Amazon_invoice4_data_extracted.xlsx'")   


Extracted Shipping Address:
 sakthivel sakthivel 42-keela vaikolkara street ,kulathu sandhu, woraiyur Trichy, TAMIL NADU, 620003 IN State/UT Code: 33 Place of supply: TAMIL NADU Place of delivery: TAMIL NADU
Extracted Billing Address    :     
CAMBIUM RETAIL PRIVATE LIMITED sakthivel
* GMR Airport City, Survey No. 99/1, Mamidipally 42-keela vaikolkara street ,kulathu sandhu,
Village, Shamshabad woraiyur
Hyderabad, Telangana, 500108 Trichy, TAMIL NADU, 620003
IN IN
State/UT Code: 33
PAN No: AAMCA5258P
GST Registration No: 36AAMCA5258P1ZY 
Extracted Product_Details    :     (Renewed) PTron Soundster Bluetooth Headphones, Over The Ear, Wireless Earphones with Mic, High Bass, 20 Hours ₹824.58 ₹0.00 1 ₹824.58 18% IGST ₹148.42 ₹973.00 of Music Time, Lightweight & Foldable Design for All Smartphones (Black)
Data saved to 'Amazon_invoice4_data_extracted.xlsx'
